In [1]:
import pandas as pd
import pickle

In [9]:
from datetime import datetime
import os

In [2]:
# Carga del pipeline 
with open('../artifacts/pipeline.pkl', 'rb') as  f:
    restaurant_model_pipeline = pickle.load(f)

# Cargar configuraciones de ingeniería de características
with open('../artifacts/feature_eng_configs.pkl', 'rb') as f:
    feature_eng_configs = pickle.load(f)

# Cargar el modelo guardado
with open('../models/random_forest_v1.pkl', 'rb') as f:
    modelo = pickle.load(f)


In [3]:
# Cargar el dataset de prueba
data_test = pd.read_csv('../data/raw/test.csv')

In [5]:
data_test.drop(["CustomerID", "PreferredCuisine", "Gender", "TimeOfVisit", "AverageSpend", "Age"], axis=1, inplace=True)

# codificación de variable MealType
data_test['MealType'] = pd.get_dummies(data_test['MealType'], drop_first=True).astype(int)

# codificación de Variables VisitFrequency y DiningOccasion
data_test['VisitFrequency'] = data_test['VisitFrequency'].map(feature_eng_configs['visit_freq_counts'])
data_test['DiningOccasion'] = data_test['DiningOccasion'].map(feature_eng_configs['dining_occ_counts'])

In [6]:
# Cargar el scaler estándar si fue usado previamente
with open('../artifacts/std_scaler.pkl', 'rb') as f:
    std_scaler = pickle.load(f)

In [7]:
# Estandarizar las variables del dataset de prueba
X_data_test_std = std_scaler.transform(data_test)

In [17]:
# Realizar predicciones
model_predicts = modelo.predict(X_data_test_std)

In [18]:
# Crear un DataFrame con las predicciones
predictions_df = pd.DataFrame(model_predicts, columns=["Prediction"])

In [19]:
# Obtener la fecha y hora actual en el formato especificado
timestamp = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [20]:
# Ruta de la subcarpeta donde se almacenarán las predicciones
output_folder = '../data/predictions'
os.makedirs(output_folder, exist_ok=True)  # Crear la carpeta si no existe

In [21]:
# Nombre del archivo basado en la fecha y hora
output_file = os.path.join(output_folder, f'predictions-{timestamp}.csv')

# Guardar las predicciones en un archivo CSV
predictions_df.to_csv(output_file, index=False)
print(f"Predicciones guardadas en: {output_file}")

Predicciones guardadas en: ../data/predictions\predictions-2024-12-17-19-58-52.csv


In [22]:
import mlflow

# Iniciar un experimento en MLflow
mlflow.set_experiment("Restaurant Predictions")

# Registrar el artefacto en MLflow
with mlflow.start_run():
    # Guardar el archivo de predicciones en MLflow
    mlflow.log_artifact(output_file, artifact_path="predictions")
    print(f"Archivo de predicciones registrado en MLflow: {output_file}")


Archivo de predicciones registrado en MLflow: ../data/predictions\predictions-2024-12-17-19-58-52.csv
